### 🎥 Cine-Match - Movie Recommendation System 📹


### File: Hybrid_Collaborative_Recommender.ipynb - CineMatch

- Contributor: Sudesh Kumar Santhosh Kumar
- Email: santhosh@usc.edu
- GitHub: [Sudesh Kumar](https://github.com/SudeshKumarSanthosh)
- Date: 6th December, 2023
- Course: EE-541 Introduction to Deep Learning
- Team:
  - Sudesh Kumar Santhosh Kumar
  - Ori Shirin

### Description:

In this innovative Python notebook `Hybrid_Collaborative_Recommender.ipynb`, we introduce the next evolution of the CineMatch movie recommendation system. This advanced system is at the forefront of recommendation technology, embracing a hybrid collaborative approach that synergizes both `implicit` and `explicit` user signals.

The core of this notebook is the development of a sophisticated recommendation engine that not only considers users' explicit feedback, such as `ratings and reviews`, but also incorporates implicit signals, like viewing history and interaction patterns. By blending these two dimensions, CineMatch aims to achieve a more nuanced and accurate understanding of user preferences.

